# Tokenizer

```{note}
在训练 LLM 之前，我们需要处理的第一步就是 Tokenization。模型无法直接理解文本，它只能处理数字。Tokenizer 的作用就是把文本变成数字序列。<BR/>
目前主流 LLM 几乎都使用 BPE (Byte Pair Encoding) 及其变体作为分词算法。
```

## BPE (Byte Pair Encoding) 算法

BPE 最初是一种数据压缩算法，后来被引入 NLP 领域用于分词。

### 核心思想
**“频率最高的相邻字符对，应该被合并成一个新的 token。”**

它从字符级别开始，不断合并出现频率最高的“字符对”，直到达到预设的词表大小（Vocabulary Size）。

### 算法流程

1.  **准备语料**：准备大规模的训练文本。
2.  **初始化**：把每个单词拆分成字符序列。例如 `"hug"` -> `["h", "u", "g"]`。初始词表包含所有基础字符。
3.  **统计频率**：统计所有相邻字符对（Bigram）在语料中出现的频率。
4.  **合并 (Merge)**：找到频率最高的字符对（例如 `"u"` 和 `"g"` 经常一起出现），将它们合并成一个新的 token `"ug"`。并把语料中所有的 `"u", "g"` 替换为 `"ug"`。
5.  **迭代**：重复步骤 3 和 4，直到词表大小达到预设值（例如 32000 或 100000）。

## 为什么 LLM 偏爱 BPE？

### 1. 解决 OOV (Out of Vocabulary) 问题
传统的按词分词（Word-based）如果遇到没见过的词（例如 `"ChatGPT"`），只能标记为 `<UNK>` (Unknown)，丢失信息。
BPE 遇到生僻词时，会自动退化为子词甚至字符。例如 `"ChatGPT"` 可能被拆解为 `["Chat", "G", "PT"]`。只要基础字符在词表里，模型就能处理任何字符串，永远不会出现 `<UNK>`。

### 2. 平衡词表大小与序列长度
- **Character-based**（按字分）：词表很小（26字母+符号），但序列极长，模型计算量大，且难以学到词义。
- **Word-based**（按词分）：序列短，但词表巨大（几十万），稀疏性严重，且容易 OOV。
- **Subword-based (BPE)**：折中方案。常见词是一个 token（如 `"apple"`），生僻词拆成多个 token（如 `"apple"`, `"sauce"`）。既保证了高频词的语义完整性，又控制了词表大小。

### 3. 适应多语言
BPE 不需要像中文分词那样依赖复杂的语法规则。它纯粹基于统计，因此可以把中文、英文、代码、数学公式混在一起训练，自动发现跨语言的通用结构（例如 HTML 标签 `<div>` 在任何语言语料里都是高频的，会被合并成一个 token）。

## BPE 实现

In [1]:
import re
import json
from collections import Counter
from tqdm import tqdm

class BPETokenizer:
    def __init__(self):
        self.merges = {}  # (p0, p1) -> new_id
        self.vocab = {}   # id -> bytes
        self.special_tokens = {} # str -> id
        self.inverse_special_tokens = {} # id -> str
        # GPT-4 风格的正则表达式
        # '(?:[sdmt]|ll|ve|re) 匹配常见缩写
        # ?\w+ 匹配单词
        # ?\d+ 匹配数字
        # ?[^\s\w\d]+ 匹配符号
        # \s+(?!\S) 匹配尾部空格
        # \s+ 匹配其他空格
        self.pattern = re.compile(r"""'(?:[sdmt]|ll|ve|re)| ?\w+| ?\d+| ?[^\s\w\d]+|\s+(?!\S)|\s+""")

    def train(self, text, vocab_size, special_tokens=None):
        """
        训练 BPE 分词器
        :param text: 训练文本
        :param vocab_size: 目标词表大小
        :param special_tokens: 特殊 token 列表，如 ["<unk>", "<pad>"]
        """
        print(f"Training BPE Tokenizer with target vocab size: {vocab_size}...")
        
        # 1. 预处理 Special Tokens 数量
        if special_tokens is None:
            special_tokens = []
        num_special_tokens = len(special_tokens)
        assert vocab_size >= 256 + num_special_tokens, "Vocab size must be at least 256 + special_tokens"
        
        # 计算 BPE 合并的目标 ID 上限
        # 我们需要保留最后的 N 个 ID 给 Special Tokens
        bpe_vocab_limit = vocab_size - num_special_tokens
        
        # 2. 预分词（Pre-tokenize）
        # 将文本切分成单词块，防止跨单词合并，例如 "dog." 中的 "g" 和 "." 不应该被合并
        text_chunks = re.findall(self.pattern, text)
        
        # 3. 统计 Chunk 频率并转换为初始字节序列
        chunk_counts = Counter(text_chunks)
        
        # ids_chunks: { "chunk_str": [byte_id1, byte_id2, ...] }
        # 初始状态下，ID 就是 0-255 的字节值
        ids_chunks = {chunk: [b for b in chunk.encode('utf-8')] for chunk in chunk_counts}
        
        # 初始化基础词表 (0-255)
        for i in range(256):
            self.vocab[i] = bytes([i])
        
        # 下一个可用的 ID (从 256 开始)
        next_id = 256
        
        # 4. 迭代合并 (Training Loop)
        num_merges = bpe_vocab_limit - 256
        with tqdm(total=num_merges, desc="Training BPE") as pbar:
            while len(self.vocab) < bpe_vocab_limit:
                # 统计当前所有 adjacent pairs 的频率
                stats = Counter()
                for chunk, freq in chunk_counts.items():
                    ids = ids_chunks[chunk]
                    for i in range(len(ids) - 1):
                        pair = (ids[i], ids[i+1])
                        stats[pair] += freq
                
                if not stats:
                    print("No more pairs to merge. Stopping early.")
                    break
                    
                # 找到频率最高的 pair
                # most_common(1) 返回 [(pair, count)]，取 [0][0] 得到 pair
                best_pair = stats.most_common(1)[0][0]
                
                # 记录合并规则
                self.merges[best_pair] = next_id
                
                # 更新词表：新 token 的字节序列 = 左 token 字节 + 右 token 字节
                self.vocab[next_id] = self.vocab[best_pair[0]] + self.vocab[best_pair[1]]
                
                # 在所有 chunks 中应用合并
                # 这是一个简单的 O(N) 实现，效率一般但逻辑清晰
                for chunk in ids_chunks:
                    ids = ids_chunks[chunk]
                    new_ids = []
                    i = 0
                    while i < len(ids):
                        # 如果发现当前位置匹配 best_pair，则合并
                        if i < len(ids) - 1 and ids[i] == best_pair[0] and ids[i+1] == best_pair[1]:
                            new_ids.append(next_id)
                            i += 2
                        else:
                            new_ids.append(ids[i])
                            i += 1
                    ids_chunks[chunk] = new_ids
                
                next_id += 1
                pbar.update(1)
        
        # 5. 处理 Special Tokens (分配最后的 ID)
        if special_tokens:
            for token in special_tokens:
                # 确保 special tokens 不会覆盖已有的 ID
                # 注意：这里我们简单地将它们作为独立的 entry 加入词表
                # 它们没有对应的 merges 规则，因为它们是不可分割的整体
                self.vocab[next_id] = token.encode('utf-8')
                self.special_tokens[token] = next_id
                self.inverse_special_tokens[next_id] = token
                next_id += 1
                
        print(f"Training complete. Final vocab size: {len(self.vocab)}")
        print(f"Special tokens map: {self.special_tokens}")

    def encode(self, text):
        """
        将文本编码为 token ids
        """
        # 1. 处理 Special Tokens
        # 如果有 special tokens，我们需要先将它们从文本中切分出来，防止被 BPE 打碎
        if not self.special_tokens:
            special_pattern = None
        else:
            # 构造一个匹配任意 special token 的正则，注意要转义，排序是为了优先匹配更长的 token
            sorted_specials = sorted(self.special_tokens.keys(), key=len, reverse=True)
            special_pattern = re.compile("|".join(re.escape(k) for k in sorted_specials))

        # 最终的 ids 列表
        ids = []
        
        # 辅助函数：对一段没有 special token 的纯文本进行 BPE 编码
        def _encode_chunk(text_chunk):
            if not text_chunk:
                return []
            
            # 1. 预分词 (Regex split)
            words = re.findall(self.pattern, text_chunk)
            chunk_ids = []
            
            for word in words:
                # 转为字节序列
                word_bytes = [b for b in word.encode('utf-8')]
                
                # 2. BPE Merge
                # 这里我们需要不断合并，直到无法合并为止
                # 这是一个简单的实现：每次扫描所有可能的 pairs，找到在 merges 中最早出现的那个进行合并
                while len(word_bytes) >= 2:
                    # 找出当前序列中所有相邻的 pair
                    stats = {}
                    for i in range(len(word_bytes) - 1):
                        pair = (word_bytes[i], word_bytes[i+1])
                        # 检查这个 pair 是否在我们的合并规则中
                        if pair in self.merges:
                            stats[pair] = self.merges[pair] # 记录 pair -> new_id
                    
                    if not stats:
                        break # 没有可以合并的 pair 了
                    
                    # 找到优先级最高（new_id 最小，即最早被 merge）的 pair
                    # BPE 的合并顺序必须严格遵循训练时的顺序
                    best_pair = min(stats, key=lambda p: self.merges[p])
                    new_id = self.merges[best_pair]
                    
                    # 执行合并
                    new_word_bytes = []
                    i = 0
                    while i < len(word_bytes):
                        if i < len(word_bytes) - 1 and word_bytes[i] == best_pair[0] and word_bytes[i+1] == best_pair[1]:
                            new_word_bytes.append(new_id)
                            i += 2
                        else:
                            new_word_bytes.append(word_bytes[i])
                            i += 1
                    word_bytes = new_word_bytes
                
                chunk_ids.extend(word_bytes)
            return chunk_ids

        # 如果没有 special tokens，直接处理
        if not special_pattern:
            return _encode_chunk(text)

        # 如果有 special tokens，我们需要切分
        start = 0
        for match in special_pattern.finditer(text):
            # 处理前面的普通文本
            non_special_text = text[start:match.start()]
            if non_special_text:
                ids.extend(_encode_chunk(non_special_text))
            
            # 处理 special token
            special_token = match.group()
            ids.append(self.special_tokens[special_token])
            
            start = match.end()
        
        # 处理剩余的文本
        remaining_text = text[start:]
        if remaining_text:
            ids.extend(_encode_chunk(remaining_text))
            
        return ids

    def decode(self, ids):
        """
        将 token ids 解码为文本
        """
        text_parts = []
        current_bytes = []
        
        for idx in ids:
            # 如果是 Special Token
            if idx in self.inverse_special_tokens:
                # 先把积攒的 bytes 解码并加入
                if current_bytes:
                    text_parts.append(b"".join(current_bytes).decode('utf-8', errors='replace'))
                    current_bytes = []
                # 加入 special token 字符串
                text_parts.append(self.inverse_special_tokens[idx])
            else:
                # 如果是普通 Token，查表得到 bytes
                # 注意：self.vocab[idx] 可能是单个字节，也可能是合并后的字节序列
                if idx in self.vocab:
                    current_bytes.append(self.vocab[idx])
                else:
                    # 未知 token (理论上不应该发生，除非 vocab 没对齐)
                    pass
        
        # 处理最后剩余的 bytes
        if current_bytes:
            text_parts.append(b"".join(current_bytes).decode('utf-8', errors='replace'))
            
        return "".join(text_parts)

    def tokenize(self, text):
        """
        将文本切分为 token 字符串列表，便于观察分词结果
        对于无法解码为有效 UTF-8 的字节序列（如被切断的中文字符），将显示其字节表示（如 b'\\xe4'）
        """
        ids = self.encode(text)
        tokens = []
        for idx in ids:
            if idx in self.inverse_special_tokens:
                tokens.append(self.inverse_special_tokens[idx])
            elif idx in self.vocab:
                token_bytes = self.vocab[idx]
                try:
                    # 尝试解码为字符串
                    tokens.append(token_bytes.decode('utf-8'))
                except UnicodeDecodeError:
                    # 如果是无效的 utf-8 序列（比如被切断的多字节字符），显示其字节表示
                    tokens.append(str(token_bytes))
            else:
                # Fallback for unknown ids
                tokens.append(f"<ID:{idx}>")
        return tokens

    def save(self, file_path):
        """
        保存模型到 JSON 文件
        """
        # merges 的 key 是 tuple，JSON 不支持，转成 list 存储
        # 格式: [ [p0, p1], new_id ]
        merges_list = [[list(pair), new_id] for pair, new_id in self.merges.items()]
        
        model_data = {
            "merges": merges_list,
            "special_tokens": self.special_tokens
        }
        
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(model_data, f, ensure_ascii=False, indent=2)

    def load(self, file_path):
        """
        从 JSON 文件加载模型
        """
        with open(file_path, 'r', encoding='utf-8') as f:
            model_data = json.load(f)
            
        # 1. 恢复 merges
        # JSON 里的 list 变成了 [ [p0, p1], new_id ]
        self.merges = {tuple(pair): new_id for pair, new_id in model_data["merges"]}
        
        # 2. 恢复 special_tokens
        self.special_tokens = model_data["special_tokens"]
        self.inverse_special_tokens = {v: k for k, v in self.special_tokens.items()}
        
        # 3. 重建 vocab
        self.vocab = {}
        # 3.1 基础字符 (0-255)
        for i in range(256):
            self.vocab[i] = bytes([i])
            
        # 3.2 根据 merges 重建组合 token
        # 必须按 new_id 从小到大顺序执行，因为后面的 token 可能依赖前面的
        sorted_merges = sorted(self.merges.items(), key=lambda item: item[1])
        for (p0, p1), new_id in sorted_merges:
            self.vocab[new_id] = self.vocab[p0] + self.vocab[p1]
            
        # 3.3 恢复 special tokens 的 bytes
        for token, idx in self.special_tokens.items():
            self.vocab[idx] = token.encode('utf-8')



## 训练和验证 BPETokenizer

In [2]:
import pandas as pd

df = pd.read_parquet('data/wikitext-103-raw-v1-train.parquet')
df

,page
0,= Valkyria Chronicles III = \n \n Senjō no Va...
1,= Tower Building of the Little Rock Arsenal =...
2,= Cicely Mary Barker = \n \n Cicely Mary Bark...
3,= Gambia women 's national football team = \n...
4,= Plain maskray = \n \n The plain maskray or ...
...,...
29439,"= Si Una Vez = \n \n "" Si Una Vez "" ( English..."
29440,= Sicklefin lemon shark = \n \n The sicklefin...
29441,= Flammulated flycatcher = \n \n The flammula...
29442,= Ontario Highway 89 = \n \n King 's Highway ...


In [3]:
import random

text = ''
for i in tqdm(range(len(df))):
    if random.random() < 0.001:
        text += df.iloc[i].to_dict()['page'] + '\n\n'
len(text)

100%|██████████| 29444/29444 [00:00<00:00, 1130946.42it/s]


835072

In [4]:
tokenizer = BPETokenizer()
tokenizer.train(text=text, vocab_size=20000, special_tokens=['<BOS>', '<EOS>', '<PAD>'])


Training BPE Tokenizer with target vocab size: 20000...


Training BPE: 100%|██████████| 19741/19741 [05:44<00:00, 57.37it/s] 

Training complete. Final vocab size: 20000
Special tokens map: {'<BOS>': 19997, '<EOS>': 19998, '<PAD>': 19999}


In [9]:
page = df.iloc[-1].to_dict()['page']
tokenizer.decode(tokenizer.encode(page)) == page

True

In [10]:
tokenizer.tokenize(page)[: 30]

[' =',
 ' Lu',
 'ke',
 ' Smith',
 ' (',
 ' writer',
 ' )',
 ' =',
 ' \n \n',
 ' Lu',
 'ke',
 ' Michael',
 ' Smith',
 ' is',
 ' an',
 ' American',
 ' writer',
 ' .',
 ' He',
 ' is',
 ' a',
 ' staff',
 ' member',
 ' at',
 ' B',
 'ung',
 'ie',
 ' ,',
 ' a',
 ' video']

In [11]:
tokenizer.save('wiki-tokenizer-1.json')

In [12]:
x = BPETokenizer()
x.load('wiki-tokenizer-1.json')
x.tokenize(page) == tokenizer.tokenize(page)


True